In [1]:
import pandas as pd
import numpy as np
import operator
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import NearestNeighbors
import quadratic_weighted_kappa
from sklearn.linear_model import Ridge
from sklearn.metrics import make_scorer
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
import feature_generator
import xgboost as xgb
from scipy import optimize
import os.path
from NN import NN
from XgBoost import XGBoostModel
from sklearn.linear_model import Ridge
from CutPoints import CutPointOptimizer

Using Theano backend.


/Library/Python/2.7/site-packages/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


In [4]:
dfTrain = pd.read_csv('train.csv')
dfTest = pd.read_csv('test.csv')  

train, test, labels = feature_generator.GetFeatures(dfTrain, dfTest, 10000, True)

featureImpDf = pd.read_csv('FeatureImportance.csv')
importantFeatures = featureImpDf[featureImpDf['Importance'] > 11]
importantFeatures = [column for column in train.columns if column in importantFeatures['Feature'].values]
importantFeatures = importantFeatures['Feature'].values
train = train[importantFeatures]
test = test[importantFeatures]

# train, test, labels = feature_generator.make_dataset(True, "mean", True, dfTrain, dfTest)

dfTrain = pd.read_csv('train.csv')
dfTest = pd.read_csv('test.csv')  

Scaling...


TypeError: list indices must be integers, not str

In [ ]:
model = XGBoostModel(900, 7, 0.025, 0.50, 25)
model.fit(train, labels, 0)

In [ ]:
importance = model.bst.get_fscore()

In [ ]:
importance = sorted(importance.items(), key=operator.itemgetter(1))
for item in importance:
    print item
# print train.shape

imp = pd.DataFrame()
imp['Feature'] = [item[0] for item in importance]
imp['Importance'] = [item[1] for item in importance]
imp.to_csv(path_or_buf='FeatureImportance.csv', index=False)

In [ ]:
def WritePredictionsToFile(model, modelName):
    
    kf = KFold(len(train), 3)
    num = 1
    for train_index, test_index in kf:
        foldFile = 'fold%s.csv' % str(num)
        if os.path.isfile(foldFile):
            predictionsDF = pd.read_csv(foldFile)  
        else:
            predictionsDF = pd.DataFrame()
          
        xTrain = train.iloc[train_index].values
        yTrain = labels.iloc[train_index]      
        model.fit(xTrain, yTrain)
        trainPredictions = model.predict(xTrain)
        
        xValidate = train.iloc[test_index].values
        yValidate = labels.iloc[test_index]
        predictions = model.predict(xValidate)
        predictionsDF[modelName] = predictions
        
        print quadratic_weighted_kappa.quadratic_weighted_kappa(predictions, yValidate)
        print quadratic_weighted_kappa.quadratic_weighted_kappa(trainPredictions, yTrain)
        
#         youngTrainPredictions = [trainPredictions[i] for i in range(len(xTrain)) if i in youngInd]
#         oldTrainPredictions = [trainPredictions[i] for i in range(len(trainPredictions)) if i in oldInd]
#         youngYTrain = yTrain.iloc[youngInd] 
                        
        predictionsDF.to_csv(path_or_buf=foldFile, index=False)
   
        predictionsFile = 'testPredictions%s.csv' % str(num)
        if os.path.isfile(predictionsFile):
            testDF = pd.read_csv(predictionsFile)   
        else:
            testDF = pd.DataFrame()
            
        xTest = test.values
        testPredictions = model.predict(xTest)
        testDF[modelName] = testPredictions
        testDF.to_csv(path_or_buf=predictionsFile, index=False)
    
        num += 1

In [ ]:
WritePredictionsToFile(LogisticRegression(), 'LogisticRegression')
# WritePredictionsToFile(XGBoostModel(700, 10, 0.025, 0.65, "reg:linear"), 'XGBoostRegLin')

# WritePredictionsToFile(XGBoostModel(0.3, 1, 0, 700), 'XGBoostLinear')
# WritePredictionsToFile(RandomForestRegressor(n_estimators=10, max_depth=10), 'RandomForest')
# WritePredictionsToFile(BaggingRegressor(base_estimator=DecisionTreeRegressor(), n_estimators=20), 'BaggingDescisionTrees_n_estimators=20')
# WritePredictionsToFile(BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=20, max_features=0.9, max_samples=1.0), 'BaggingDescisionTreeClassifiers_n_estimators=20')
# WritePredictionsToFile(BaggingRegressor(base_estimator=LinearRegression(), n_estimators=10), 'BaggingLinearRegression_n_estimators=10')

In [ ]:
dfTrain = pd.read_csv('train.csv')
dfTest = pd.read_csv('test.csv')    
# train, test, labels = feature_generator.GetFeatures(dfTrain, dfTest, 100)
train, test, labels = feature_generator.make_dataset(True, "mean", True, dfTrain, dfTest)
WritePredictionsToFile(NN(inputShape = train.shape[1], layers = [128, 64], dropout = [0.5, 0.5], activation='sigmoid', loss='mae', optimizer = 'adadelta', init = 'glorot_normal', nb_epochs = 8), 'Keras')

In [ ]:

scorer = make_scorer(quadratic_weighted_kappa.quadratic_weighted_kappa)
# print len(features)
# print len(dummyVariables)

In [ ]:
def qwk_scorer(estimator, X, Y):
    predictions = np.clip(estimator.predict(X), 1, 8)
    return quadratic_weighted_kappa.quadratic_weighted_kappa(predictions, Y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train, labels, test_size=0.20, random_state=0)
bcf = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=20, max_features=0.9, max_samples=1.0)
bcf.fit(X_train, y_train)
# Set the parameters by cross-validation
tuned_parameters = [{'n_estimators': [15, 20, 25], 'max_samples': [0.9, 1.0], 'max_features': [0.9, 1.0]}]
# clf = GridSearchCV(SVR(kernel='rbf', max_iter=1000, epsilon=0.49, tol=0.01, verbose=True), tuned_parameters, cv=3, scoring=qwk_scorer)
# clf = GridSearchCV(BaggingClassifier(base_estimator=DecisionTreeClassifier()), tuned_parameters, cv=3, scoring=qwk_scorer)
# clf.fit(X_train, y_train)
# SVR(kernel='rbf', max_iter=1, , tol=0.01, verbose=True)  
#C=5, g=0.1
# TestQWK: 0.24807920235173586
#C=2, g=0.05, qwk = 0.21
#C=5, g=0.05, qwk = 0.25465662455597715
# do C 

In [ ]:
print clf.best_estimator_

In [ ]:
bcp = bcf.predict(X_test)
print quadratic_weighted_kappa.quadratic_weighted_kappa(bcp, y_test)

In [ ]:
print clf.best_params_
clfPredictions = np.clip(clf.predict(X_test), 1, 8)

# dataPoints = list()

# folds = (2, 5, 10, 20)
# for K in folds:
#     _, testQwk, trainQwk = GetBestModel(GenerateNewCombinedModel, features, K)
#     dataPoints.append((K, testQwk, trainQwk))

In [ ]:
print np.min(clfPredictions)
print np.max(clfPredictions)
print clfPredictions

In [ ]:
import theano
import theano.tensor as T
from keras import backend as K

def custom_loss(y_true, y_pred):
    
#     medianCutPoints = theano.shared(np.array([2.00711375,  3.37756859,  4.31663661,  5.32317815,  6.09457793,  6.81579768, 7.52165309]))
    x = K.abs(y_true - y_pred)
    return K.mean(K.maximum(x-K.exp(-6*x), 0.0), axis=-1)
#     return K.mean((x)*(1-K.exp(-(x))), axis=-1)
#     return K.mean(K.maximum(x-0.25, 0.0), axis=-1)

In [7]:
from keras import objectives

# X_train, X_test, y_train, y_test = train_test_split(train, labels, test_size=0.20, random_state=1892)
X_train = train
y_train = labels

folds = 4
kf = KFold(len(X_train), folds)
num = 1
combinedModels = list()
qwks = list()

cpa = np.ndarray((5, 7))
# cpa[0,:] = [1.94094589,  2.77827061, 4.47922564,  5.59375133,  5.98938611,  6.86875996, 7.45790894]
cpa[0,:] = [2.27420049,  3.32644001,  3.22110338,  5.37172015,  6.01216561,  6.68318259, 7.4242825]
cpa[1,:] = [1.93043217,  3.06875824,  4.8242938,   5.40717815,  6.17241788,  6.97729218, 7.46620894]
cpa[2,:] = [1.23315182,  3.39688586,  4.6380404,   5.37977014,  6.16224374,  6.73081194, 7.48322552]
cpa[3,:] = [2.33061803,  3.5055729,   4.52173829,  5.24184554,  6.1711789,   6.96746293, 7.7211743]
cpa[4,:] = [2.26716625,  3.59018596,  4.3780072,   5.21537676,  5.95488354,  6.72023874, 7.5133742]

# nnModel = NN(inputShape = train.shape[1], cutPointArray=cpa, layers = [250, 75, 25], dropout = [0.5, 0.5, 0.5], activation='relu', patience=5, loss=custom_loss, optimizer = 'adadelta', init = 'glorot_normal', nb_epochs = 50)

for train_index, test_index in kf:
    
    combinedModel = CombinedModel([], [XGBoostModel(700, 7, 0.025, 0.50, 25)])
#     combinedModel = CombinedModel(['Boost'], [])
    
    trainFile = 'combinedTrainPredictions%s.csv' % str(num)
    validateFile = 'combinedValidatePredictions%s.csv' % str(num)

    xTrain = X_train.iloc[train_index].values
    yTrain = y_train.iloc[train_index]      
    xValidate = X_train.iloc[test_index].values
    yValidate = y_train.iloc[test_index]

    combinedModel.fit(xTrain, yTrain, trainFile, num-1)
    predictions = combinedModel.predict(xValidate, validateFile, num-1)
    qwk = quadratic_weighted_kappa.quadratic_weighted_kappa(predictions, yValidate)
    qwks.append(qwk)
    print qwk
    
    combinedModels.append(combinedModel)
    num += 1
    
meanQwk = quadratic_weighted_kappa.mean_quadratic_weighted_kappa(qwks)
print "Overall Test Qwk: %s" % meanQwk

[0]	train-kappa:-0.000000
[1]	train-kappa:-0.000000
[2]	train-kappa:-0.000000
[3]	train-kappa:-0.000000
[4]	train-kappa:-0.000000
[5]	train-kappa:-0.000000
[6]	train-kappa:-0.000000
[7]	train-kappa:-0.000000
[8]	train-kappa:-0.000000
[9]	train-kappa:-0.027943
[10]	train-kappa:-0.043175
[11]	train-kappa:-0.053980
[12]	train-kappa:-0.056978
[13]	train-kappa:-0.055907
[14]	train-kappa:-0.053054
[15]	train-kappa:-0.049413
[16]	train-kappa:-0.045879
[17]	train-kappa:-0.041538
[18]	train-kappa:-0.038216
[19]	train-kappa:-0.034890
[20]	train-kappa:-0.032050
[21]	train-kappa:-0.045942
[22]	train-kappa:-0.064824
[23]	train-kappa:-0.074395
[24]	train-kappa:-0.081703
[25]	train-kappa:-0.088135
[26]	train-kappa:-0.092916
[27]	train-kappa:-0.096353
[28]	train-kappa:-0.098487
[29]	train-kappa:-0.099430
[30]	train-kappa:-0.100089
[31]	train-kappa:-0.107694
[32]	train-kappa:-0.129702
[33]	train-kappa:-0.148321
[34]	train-kappa:-0.158889
[35]	train-kappa:-0.166418
[36]	train-kappa:-0.172615
[37]	train-

Optimization terminated successfully.
         Current function value: -0.750802
         Iterations: 140
         Function evaluations: 283
Train QWK: 0.7508021494

0.65347253962

[0]	train-kappa:-0.000000
[1]	train-kappa:-0.000000
[2]	train-kappa:-0.000000
[3]	train-kappa:-0.000000
[4]	train-kappa:-0.000000
[5]	train-kappa:-0.000000
[6]	train-kappa:-0.000000
[7]	train-kappa:-0.000000
[8]	train-kappa:-0.000000
[9]	train-kappa:-0.027221
[10]	train-kappa:-0.044445
[11]	train-kappa:-0.054444
[12]	train-kappa:-0.057209
[13]	train-kappa:-0.055807
[14]	train-kappa:-0.053280
[15]	train-kappa:-0.049959
[16]	train-kappa:-0.046134
[17]	train-kappa:-0.042143
[18]	train-kappa:-0.038419
[19]	train-kappa:-0.034509
[20]	train-kappa:-0.031719
[21]	train-kappa:-0.037911
[22]	train-kappa:-0.064180
[23]	train-kappa:-0.073492
[24]	train-kappa:-0.080168
[25]	train-kappa:-0.087199
[26]	train-kappa:-0.092708
[27]	train-kappa:-0.096091
[28]	train-kappa:-0.098282
[29]	train-kappa:-0.099380
[30]	train-kappa:-0.100208
[31]	train-kappa:-0.106030
[32]	train-kappa:-0.126998
[33]	train-kappa:-0.149147
[34]	train-kappa:-0.160082
[35]	train-kappa:-0.166515
[36]	train-kappa:-0.172198
[37]	train-


Optimization terminated successfully.
         Current function value: -0.753574
         Iterations: 151
         Function evaluations: 293
Train QWK: 0.753574258641

0.655978714789

[0]	train-kappa:-0.000000
[1]	train-kappa:-0.000000
[2]	train-kappa:-0.000000
[3]	train-kappa:-0.000000
[4]	train-kappa:-0.000000
[5]	train-kappa:-0.000000
[6]	train-kappa:-0.000000
[7]	train-kappa:-0.000000
[8]	train-kappa:-0.000000
[9]	train-kappa:-0.024116
[10]	train-kappa:-0.044916
[11]	train-kappa:-0.054085
[12]	train-kappa:-0.056635
[13]	train-kappa:-0.055230
[14]	train-kappa:-0.051927
[15]	train-kappa:-0.048464
[16]	train-kappa:-0.044694
[17]	train-kappa:-0.040952
[18]	train-kappa:-0.037246
[19]	train-kappa:-0.033488
[20]	train-kappa:-0.030613
[21]	train-kappa:-0.041266
[22]	train-kappa:-0.061067
[23]	train-kappa:-0.072003
[24]	train-kappa:-0.079174
[25]	train-kappa:-0.084486
[26]	train-kappa:-0.089894
[27]	train-kappa:-0.094119
[28]	train-kappa:-0.097003
[29]	train-kappa:-0.097975
[30]	train-kappa:-0.098706
[31]	train-kappa:-0.105630
[32]	train-kappa:-0.125041
[33]	train-kappa:-0.146317
[34]	train-kappa:-0.158076
[35]	train-kappa:-0.165851
[36]	train-kappa:-0.171192
[37]	train-


Optimization terminated successfully.
         Current function value: -0.751779
         Iterations: 139
         Function evaluations: 265
Train QWK: 0.751779432643

0.655458547729

[0]	train-kappa:-0.000000
[1]	train-kappa:-0.000000
[2]	train-kappa:-0.000000
[3]	train-kappa:-0.000000
[4]	train-kappa:-0.000000
[5]	train-kappa:-0.000000
[6]	train-kappa:-0.000000
[7]	train-kappa:-0.000000
[8]	train-kappa:-0.000000
[9]	train-kappa:-0.024398
[10]	train-kappa:-0.044896
[11]	train-kappa:-0.054728
[12]	train-kappa:-0.056639
[13]	train-kappa:-0.055506
[14]	train-kappa:-0.052512
[15]	train-kappa:-0.048715
[16]	train-kappa:-0.044858
[17]	train-kappa:-0.041095
[18]	train-kappa:-0.037529
[19]	train-kappa:-0.033855
[20]	train-kappa:-0.030977
[21]	train-kappa:-0.040372
[22]	train-kappa:-0.060609
[23]	train-kappa:-0.071529
[24]	train-kappa:-0.077859
[25]	train-kappa:-0.084445
[26]	train-kappa:-0.091077
[27]	train-kappa:-0.094818
[28]	train-kappa:-0.097335
[29]	train-kappa:-0.098253
[30]	train-kappa:-0.099384
[31]	train-kappa:-0.099289
[32]	train-kappa:-0.123201
[33]	train-kappa:-0.145475
[34]	train-kappa:-0.157321
[35]	train-kappa:-0.165394
[36]	train-kappa:-0.171281
[37]	train-


Optimization terminated successfully.
         Current function value: -0.747403
         Iterations: 115
         Function evaluations: 239
Train QWK: 0.747403149753

0.660406480846
Overall Test Qwk: 0.65633649509


In [ ]:
# validatePredictions = np.zeros(len(X_test))
cutPoints = np.ndarray((7, 7))
coef = 0
inter = 0
for i in range(1, folds + 1):
    model = combinedModels[i-1]
#     print model.cutPoints
    coef += model.stackingModel.coef_ / folds
    inter += model.stackingModel.intercept_ / folds
    cutPoints[i-1] = model.cutPoints
    
print coef
print inter
    
# print np.mean(cutPoints, axis=0)
#     validatePredictions += model.predict(X_test, 'combinedTestPredictions%s.csv' % str(i), i)
    
# validatePredictions /= folds
# validatePredictions = np.round(validatePredictions).astype(int)

# print quadratic_weighted_kappa.quadratic_weighted_kappa(validatePredictions, y_test)

In [5]:
class CombinedModel:
    
    def __init__(self, modelsFromFile, modelsToCalculate):
        self.modelsToCalculate = modelsToCalculate 
        self.modelsFromFile = modelsFromFile
        self.stackingModel = LinearRegression()
        self.boost = [0.652626727963, 0.652281118598, 0.651128156224, 0.651189716136, 0.662062433355]
        self.keras = [0.621456525096, 0.640965818589, 0.64586691993, 0.66517675706, 0.679622912852]
        
    def fit(self, X, Y, fileName, num):     
        
        stackingData = np.ndarray((X.shape[0], len(self.modelsFromFile) + len(self.modelsToCalculate)))    
        df = pd.read_csv(fileName) if os.path.isfile(fileName) else pd.DataFrame()
        
        for i in range(len(self.modelsToCalculate)):
            model = self.modelsToCalculate[i]
            model.fit(X, Y, num)
            predictions = model.predict(X)
            stackingData[:,i] = predictions
#             df[model] = predictions
            
        if len(self.modelsFromFile) > 0:
            colsToChange = range(len(self.modelsToCalculate), len(self.modelsToCalculate) + len(self.modelsFromFile))
            stackingData[:,colsToChange] = df[self.modelsFromFile].values
            
        self.stackingModel.fit(stackingData, Y)
        predictions = self.stackingModel.predict(stackingData)
#         predictions = np.mean(stackingData, axis=1)
#         predictions = np.add(self.boost[num] * stackingTrainData[:,0], self.keras[num]*stackingTrainData[:,1]) / (self.boost[num] + self.keras[num])
        
        initialCutPoints = np.array([1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5])
        cpo = CutPointOptimizer(predictions, Y)
        self.cutPoints = optimize.fmin(cpo.qwk, initialCutPoints)
        
        predictions = np.searchsorted(self.cutPoints, predictions) + 1   

        trainQwk = quadratic_weighted_kappa.quadratic_weighted_kappa(predictions, Y)
        print "Train QWK: %s\n" % trainQwk
                           
    def predict(self, X, fileName, num):
        stackingData = np.ndarray((X.shape[0], len(self.modelsFromFile) + len(self.modelsToCalculate)))
        df = pd.read_csv(fileName) if os.path.isfile(fileName) else pd.DataFrame()
        
        for i in range(len(self.modelsToCalculate)):
            model = self.modelsToCalculate[i]
            predictions = model.predict(X)
            stackingData[:,i] = predictions
            
        if len(self.modelsFromFile) > 0:
            colsToChange = range(len(self.modelsToCalculate), len(self.modelsToCalculate) + len(self.modelsFromFile))
            stackingData[:,colsToChange] = df[self.modelsFromFile].values
            
        predictions = self.stackingModel.predict(stackingData)
#         predictions = np.mean(stackingData, axis=1)
        return np.searchsorted(self.cutPoints, predictions) + 1      
#         return np.searchsorted(np.median(cpa, axis=0), predictions) + 1      

In [6]:
testPredictions = np.zeros(len(test))
for i in range(1, folds + 1):
    model = combinedModels[i-1]
    testPredictions += model.predict(test, 'combinedTestPredictions%s.csv' % str(i), i)

    
testPredictions /= folds
predDf = pd.DataFrame()
predDf['Id'] = dfTest['Id']
predDf['Response'] = np.round(testPredictions).astype(int)
print predDf['Response'].values
predDf.to_csv(path_or_buf='XgBoost.csv', columns=['Id', 'Response'], index=False, header=['Id', 'Response'])

NameError: name 'folds' is not defined

In [ ]:
kf = KFold(len(train), 8)
num = 1
qwks = list()

name = 'Boost'
# model = NN(inputShape = train.shape[1], cutPointArray=cpa, layers = [250, 75, 15], dropout = [0.5, 0.5, 0.5], activation='relu', patience=5, loss='mae', optimizer = 'adadelta', init = 'glorot_normal', nb_epochs = 50)

model = XGBoostModel(800, 7, 0.025, 0.50, 25)

for train_index, test_index in kf:
    trainFile = 'combinedTrainPredictions%s.csv' % str(num)
    validateFile = 'combinedValidatePredictions%s.csv' % str(num)
    testFile = 'combinedTestPredictions%s.csv' % str(num)
    
    trainDF = pd.read_csv(trainFile) if os.path.isfile(trainFile) else pd.DataFrame()  
    validateDF = pd.read_csv(validateFile) if os.path.isfile(validateFile) else pd.DataFrame()  
    testDF = pd.read_csv(testFile) if os.path.isfile(testFile) else pd.DataFrame()  

    xTrain = train.iloc[train_index].values
    yTrain = labels.iloc[train_index]      
    xValidate = train.iloc[test_index].values
    yValidate = labels.iloc[test_index]
    
    model.fit(xTrain, yTrain, num-1)
    testPredictions = model.predict(test.values)
    trainPredictions = model.predict(xTrain)
    validatePredictions = model.predict(xValidate)
    
    trainDF[name] = trainPredictions
    validateDF[name] = validatePredictions
    testDF[name] = testPredictions
        
    trainDF.to_csv(path_or_buf=trainFile, index=False)
    validateDF.to_csv(path_or_buf=validateFile, index=False)
    testDF.to_csv(path_or_buf=testFile, index=False)
            
    num += 1

In [ ]:
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

keywords = ['Medical_Keyword_' + str(i) for i in range(1, 49)]
# print dfTrain[keywords].sum(axis=1)

print dfTrain[]['Response'].mean()
print dfTrain['Response'].mean()
#     Medical_History_10
# # uniqueValues = dfTest['InsuredInfo_7'].unique()
# # for i in range(len(uniqueValues)):
# #     arr = dfTrain['InsuredInfo_7'].apply(lambda x: x == uniqueValues[i])
    

# for column in ['Product_Info_4', 'Ins_Age', 'Ht', 'Wt', 'BMI', 'Employment_Info_1', 'Employment_Info_4', 'Employment_Info_6', 'Insurance_History_5', 'Family_Hist_2', 'Family_Hist_3', 'Family_Hist_4', 'Family_Hist_5']:
   
#     if dfTrain[column].isnull().sum():
#         print column
#         print pearsonr(dfTrain[dfTrain[column].notnull()][column], dfTrain[dfTrain[column].notnull()].Response)
#         print dfTrain[column].median()
#         print len(dfTrain[dfTrain.Response == 8])
#         print len(dfTrain)
# #         print dfTrain[column]
# #         plt.plot(dfTrain[dfTrain[column].notnull()][column], dfTrain[dfTrain[column].notnull()].Response)
#         break
# # plt.show()
# # print dfTest['InsuredInfo_4'].unique()
# # print dfTrain['InsuredInfo_6'].apply(lambda x: x == 1)
# print dfTest['Medical_History_32'].isnull().sum()
            
# print dfTrain['Medical_History_32'].median()
# print dfTrain[dfTrain['Family_Hist_2'].notnull()]['Family_Hist_2'].max()
# # print dfTrain['Medical_History_32'].null().sum()
# # print dfTest[dfTest['Medical_History_32'].notnull()]['Medical_History_32'].median()
# # print len(pd.concat([dfTest[dfTest['Medical_History_32'].notnull()]['Medical_History_32'], dfTrain[dfTrain['Medical_History_32'].notnull()]['Medical_History_32']]))
# # print pearsonr(dfTrain[dfTrain['Medical_History_32'].notnull()]['Medical_History_32'], dfTrain[dfTrain['Medical_History_32'].notnull()]['Response'])

In [ ]:
neighborModel = GetBestModel(lambda: NearestNeighbors(), features)

In [ ]:
nearestNeighbors = neighborModel.kneighbors(dfTrain[features].values)

In [ ]:
predictions = np.zeros(len(dfTrain))
for i in range(len(dfTrain)):
#     predictions[i] = dfTrain.iloc[nearestNeighbors[1][i]].Response.mean()
    responses = dfTrain.iloc[nearestNeighbors[1][i]].Response
    weights = nearestNeighbors[0][i][4] - nearestNeighbors[0][i]
    predictions[i] = np.sum(responses * weights) / np.sum(weights)

In [ ]:
predictions = np.rint(predictions)
print quadratic_weighted_kappa.quadratic_weighted_kappa(predictions, dfTrain.Response)
print np.min(predictions)

In [ ]:
import scipy.stats as stats
lowerBound = np.mean(dfTrain['BMI']) - np.std(dfTrain['BMI'])
upperBound = np.mean(dfTrain['BMI']) + np.std(dfTrain['BMI'])
mean = np.mean(dfTrain['BMI'])
stats.normaltest(dfTrain['BMI'])

bmiScore = dfTrain['BMI'].apply(lambda x: max(0, lowerBound - x) if x < mean else max(0, x - upperBound))

model = LinearRegression(normalize=True)
features = ['Ins_Age', 'Wt', 'BMI']
X = np.ndarray((59381, 9))
X[:,0] = dfTrain['Ins_Age'].values
X[:,1] = dfTrain['BMI'].values
X[:,2] = dfTrain['Wt'].values
X[:,3] = dfTrain['Ins_Age'] * dfTrain['Wt']
X[:,4] = dfTrain['Ins_Age'] * dfTrain['BMI']
X[:,5] = dfTrain['Ins_Age'] * dfTrain['Ins_Age']
X[:,6] = dfTrain['Wt'] * dfTrain['Wt']
X[:,7] = dfTrain['Wt'] * dfTrain['BMI']
X[:,8] = dfTrain['Wt'] * dfTrain['Ins_Age']

model.fit(X, dfTrain['Response'])

In [ ]:
import theano
import theano.tensor as T
import quadratic_weighted_kappa
y1, y2 = T.vectors('y1', 'y2')
# s = 1 / (1 + T.exp(-x))
s = quadratic_weighted_kappa.quadratic_weighted_kappa(y1, y2)
# logistic = theano.function([y1, y2], s)

In [ ]:
#2 layer 200/75 cut points
cpa = np.ndarray((5, 7))
cpa[0,:] = [2.27420049,  3.32644001,  3.22110338,  5.37172015,  6.01216561,  6.68318259, 7.4242825]
cpa[1,:] = [1.93043217,  3.06875824,  4.8242938,   5.40717815,  6.17241788,  6.97729218, 7.46620894]
cpa[2,:] = [1.23315182,  3.39688586,  4.6380404,   5.37977014,  6.16224374,  6.73081194, 7.48322552]
cpa[3,:] = [2.33061803,  3.5055729,   4.52173829,  5.24184554,  6.1711789,   6.96746293, 7.7211743]
cpa[4,:] = [2.26716625,  3.59018596,  4.3780072,   5.21537676,  5.95488354,  6.72023874, 7.5133742]

print np.mean(cpa, axis=0)

In [ ]:
import theano
import theano.tensor as T

epsilon = 1.0e-9
def custom_objective(y_true, y_pred):
    y_pred
    '''Just another crossentropy'''
    y_pred = T.clip(y_pred, epsilon, 1.0 - epsilon)
    y_pred /= y_pred.sum(axis=-1, keepdims=True)
    cce = T.nnet.categorical_crossentropy(y_pred, y_true)
    return cce

In [ ]:

results, updates = theano.scan(lambda v: np.searchsorted(cutPoints, v), sequences=y_pred)
compute_elementwise = theano.function(inputs=[X, W, b_sym], outputs=[results])

result, updates = theano.scan(fn=lambda prior_result, A: np.searchsorted(cutPoints, A)
                              outputs_info=T.ones_like(A),
                              non_sequences=A,
                              n_steps=7)

# We only care about A**k, but scan has provided us with A**1 through A**k.
# Discard the values that we don't care about. Scan is smart enough to
# notice this and not waste memory saving them.
prediction = 1
for i in range(len(result)):
    if num < result[i]:
        break
    num += 1

# compiled function that returns A**k

power = theano.function(inputs=[A,k], outputs=prediction, updates=updates)

